<a href="https://colab.research.google.com/github/rodrigourech/ivi/blob/main/data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y lftp
!lftp -u seifert_FHNW,'iB7843(F' -e "set ftp:ssl-force true; set ssl:verify-certificate no; \
cd QMstations/tas; \
get CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip; \
get CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip; \
cd ../pr; \
get CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip; \
get CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip; \
quit" ftp://climftp.cscs.ch

#  Unzip all files into ./data/
!mkdir -p data
!unzip -o CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip -d ./data/
!unzip -o CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip -d ./data/
!unzip -o CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip -d ./data/
!unzip -o CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip -d ./data/

#  Combine data by scenario and variable
import pandas as pd
import os

# Selected stations (you can expand this list)
stations = {
    "BER": "Bern",
    "ZER": "Zürich",
    "LUG": "Lugano",
    "DAV": "Davos",
    "BAS": "Basel",
    "GEN": "Genf",
    "SIO": "Sion",
    "STG": "St. Gallen",
    "PAY": "Payerne"
}

scenarios = ["RCP45", "RCP85"]
variables = ["tas", "pr"]
model = "CLMCOM-CCLM4_ECEARTH_EUR11"

# Dictionary to store 4 DataFrames
df_dict = {
    "RCP45_tas": [],
    "RCP85_tas": [],
    "RCP45_pr": [],
    "RCP85_pr": []
}

# Loop over all combinations
for var in variables:
    for rcp in scenarios:
        base = f"CH2018_{var}_{model}_{rcp}_QMstations_1981-2099"
        key = f"{rcp}_{var}"
        for code, name in stations.items():
            file = f"data/{base}_{code}.csv"
            if os.path.exists(file):
                df = pd.read_csv(file, delimiter=";", skiprows=16, parse_dates=["DATE"])
                df.columns = ["date", "value"]
                df["station"] = name
                df["station_code"] = code
                df["variable"] = var
                df["scenario"] = rcp
                df["model"] = model
                df_dict[key].append(df)


# Export the 4 combined CSVs
for key, df_list in df_dict.items():
    if df_list:
        combined = pd.concat(df_list, ignore_index=True)
        combined.to_csv(f"data/{key}.csv", index=False, encoding="utf-8-sig")
        print(f"Saved: data/{key}.csv")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lftp is already the newest version (4.9.2-1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
cd ok, cwd=/QMstations/tas
get: /content/CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip: file already exists and xfer:clobber is unset
get: /content/CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip: file already exists and xfer:clobber is unset
cd ok, cwd=/QMstations/pr
